# First try to learn Logistic Regression with Neutral Network

        This is my implement of logistic regression with a neutral network. 

### Math and materials 

   All materials come from Andrew Y. Ng course.
   
   [COURSE](www.coursera.org/specializations/deep-learning)
    
### Dataset

   This is a dataset with sms spam collection from Kaggle competition 
   
   [LINK](https://www.kaggle.com/uciml/sms-spam-collection-dataset/data)
   The files contain one message per line. Each line is composed by two columns: v1 contains the label (ham or spam) and v2 contains the raw text.
   

# Overview 

First take a look on dataset, convert words to numerical expression
and change classifier to 0 when SMS is spam, 1 when SMS is ham.


In [1]:
# import useful package
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 


In [7]:
df = pd.read_csv('spam.csv', encoding= "ISO-8859-1")

df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
del(df[1:])

TypeError: 'slice(1, None, None)' is an invalid key